<a href="https://colab.research.google.com/github/weagan/Tools/blob/main/hf_to_github_transfer_(1)_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Files from Hugging Face to GitHub

This notebook transfers files from `https://huggingface.co/wtfmahe/Samsung-TRM` to `https://github.com/weagan/Samsung-TRM`

**Prerequisites:**
- Add your GitHub Personal Access Token (PAT) to Colab Secrets with the key `GITHUB_PAT`
- Go to: Secrets (🔑 icon in left sidebar) → Add new secret → Name: `GITHUB_PAT`, Value: your token

In [ ]:
# Install required packages
!pip install -q huggingface_hub

In [ ]:
# Import libraries and get GitHub PAT from secrets
from google.colab import userdata
import os

# Get GitHub PAT from Colab secrets
try:
    GITHUB_PAT = userdata.get('GITHUB_PAT')
    print("✓ GitHub PAT retrieved successfully")
except Exception as e:
    print("❌ Error: Could not retrieve GITHUB_PAT from secrets")
    print("Please add your GitHub Personal Access Token to Colab Secrets with key 'GITHUB_PAT'")
    raise e

In [ ]:
# Configure git
!git config --global user.email "colab@example.com"
!git config --global user.name "Colab User"
print("✓ Git configured")

In [ ]:
# Clone the Hugging Face repository
print("Cloning from Hugging Face...")
!git clone https://huggingface.co/wtfmahe/Samsung-TRM
print("✓ Repository cloned successfully")

In [ ]:
# Change to repository directory
%cd Samsung-TRM

# Show current remotes
print("Current remotes:")
!git remote -v

In [ ]:
# Remove Hugging Face remote and add GitHub remote
print("Removing Hugging Face remote...")
!git remote remove origin

print("Adding GitHub remote...")
# Use PAT in the URL for authentication
github_url = f"https://{GITHUB_PAT}@github.com/weagan/Samsung-TRM.git"
!git remote add origin {github_url}

print("\n✓ Remote updated successfully")
print("New remote (credentials hidden):")
!git remote -v | sed 's/:.*@/:***@/'

In [ ]:
# Check current branch
print("Current branch:")
!git branch

# Ensure we're on main branch
!git branch -M main
print("\n✓ Switched to main branch")

In [ ]:
# Check if remote has content and handle accordingly
print("Checking remote repository status...\n")
!git fetch origin main 2>&1 || echo "Remote branch does not exist yet"

# Try to get remote branch info
import subprocess
result = subprocess.run(['git', 'ls-remote', '--heads', 'origin', 'main'],
                       capture_output=True, text=True)

if result.stdout.strip():
    print("⚠️ Remote repository has existing content.")
    print("Pulling and merging changes...\n")
    !git pull origin main --allow-unrelated-histories --no-edit
    print("\nPushing to GitHub...\n")
    !git push -u origin main
else:
    print("Remote repository is empty. Pushing...\n")
    !git push -u origin main

print("\n✅ Transfer completed successfully!")
print("Your files are now available at: https://github.com/weagan/Samsung-TRM")

## Alternative: Force Push (Use with Caution)

If you want to completely overwrite the GitHub repository content with the Hugging Face content, you can run this cell instead:

**⚠️ WARNING: This will delete any existing content in the GitHub repository!**

In [ ]:
# FORCE PUSH - Only use if you want to overwrite GitHub content completely
print("⚠️ Force pushing to GitHub (this will overwrite existing content)...\n")
!git push -u origin main --force

print("\n✅ Force push completed!")
print("Your files are now available at: https://github.com/weagan/Samsung-TRM")

## Troubleshooting

If you encounter errors:

1. **Authentication failed**: Ensure your GitHub PAT has `repo` scope permissions
2. **Repository doesn't exist**: Create the repository at https://github.com/weagan/Samsung-TRM first
3. **Large files**: The notebook automatically handles Git LFS files